In [ ]:
import os
import pandas as pd
import numpy as np
from cycler import cycler
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


FILENAME = 'results/results7.csv'
TESTDIR = 'tests/test7_unfix_c6'
PLOTDIR = 'tests/test7_unfix_c6/plots'
FORMAT = 'png'

# Create target Directory if don't exist
if not os.path.exists(PLOTDIR):
    os.mkdir(PLOTDIR)
    print("Directory " , PLOTDIR ,  " Created ")

In [ ]:
''' Merge csv files '''

dlist = []
dirs = [d for d in os.listdir(TESTDIR) if os.path.isdir(os.path.join(TESTDIR,d))]
for d in dirs:
	d = os.path.abspath(TESTDIR+'/'+d)
	dlist += [os.path.join(d,file) for file in os.listdir(d) if file.endswith(".csv")]

df = pd.read_csv(dlist[0], skipinitialspace=True)
for x in range(1,len(dlist)):
    df = df.append(pd.read_csv(dlist[x], skipinitialspace=True))
df.to_csv(FILENAME,index=False)

In [ ]:
''' Mean of energy value per iteration '''

filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfe = pd.read_csv(TESTDIR+'/'+filename+'_energy.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

xticks = 0
gs = gridspec.GridSpec(4, 2)
fig = plt.figure(figsize=(15,15))
axs = []
axs += [plt.subplot(gs[0, :])] # [0] is for actual results
axs += [plt.subplot(gs[1, :])] # [1] is for zoomed in area
axs += [plt.subplot(gs[2, :])] # [2] same for failures
axs += [plt.subplot(gs[3, :])] # [3] is for zoomed in area

# Total 
for method in methods: # for each method keep structures from both dfs

    total_m = df.loc[(df['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfe_m = dfe[dfe['method'] == method] # results for this method 
    dfe_m = dfe_m.set_index('structure')
    e_res = dfe_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[0].plot(mean['energy'], label=method)
    axs[0].legend()
    
    axs[1].plot(mean['energy'], label=method)
    axs[1].legend()
    
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[2].plot(mean['energy'], label=method)
    axs[2].legend()
    
    axs[3].plot(mean['energy'], label=method)
    axs[3].legend()
 
axs[0].grid()
axs[0].set_title('Iterations of Successful Optimisation')
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[0].get_ylim()

axs[1].grid()
axs[1].set_ylim(starty, -300)
axs[1].set_xlim(0, 125)
axs[1].set_xticks(np.arange(0, 125, 5))

axs[2].grid()
axs[2].set_xlim(0, xticks+5)
axs[2].set_xticks(np.arange(0, xticks+5, 25))
axs[2].set_title('Iterations of Failed Optimisation')

axs[3].grid()
axs[3].set_ylim(-500000, 20000)
axs[3].set_xlim(0, 75)
axs[3].set_xticks(np.arange(0, 75, 5))

fig.text(-0.05, 2.5, 'Energy (eV)', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.2, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)

fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Energy Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanEnergyIterI', format=FORMAT)
plt.show()


In [ ]:
''' Mean of energy value per iteration '''

# %pylab
filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfe = pd.read_csv(TESTDIR+'/'+filename+'_energy.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

fig, axs = plt.subplots(3, 1, figsize=(15,15))

# Random 
random = df[df['structure'].str.contains(
        "rat") == False]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = random.loc[(random['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfe_m = dfe[dfe['method'] == method] # results for this method 
    dfe_m = dfe_m.set_index('structure')
    e_res = dfe_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[0].plot(mean['energy'], label=method)
    axs[0].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[1].plot(mean['energy'], label=method)
    axs[1].legend()
    
axs[0].grid()
axs[0].set_title('Successful Random')
# axs[0].set_ylim(starty, -300)
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[0].get_ylim()

axs[1].grid()
axs[1].set_xlim(0, xticks+5)
axs[1].set_xticks(np.arange(0, xticks+5, 25))
axs[1].set_title('Failed Random')


# Rattled
rattled = df[df['structure'].str.contains(
        "rat") == True]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfe_m = dfe[dfe['method'] == method] # results for this method 
    dfe_m.set_index('structure')
    e_res = dfe_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[2].plot(mean['energy'], label=method)
    axs[2].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
axs[2].grid()
axs[2].set_title('Rattled')
axs[2].set_xlim(0, xticks+5)
axs[2].set_xticks(np.arange(0, xticks+5, 5))
starty, endy = axs[2].get_ylim()

fig.text(-0.05, 1.7, 'Energy (eV)', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Energy Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanEnergyIterII', format=FORMAT)
plt.show()

In [ ]:
''' Mean of energy value per iteration '''
''' Log scale '''

# %pylab
filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfe = pd.read_csv(TESTDIR+'/'+filename+'_energy.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

fig, axs = plt.subplots(3, 1, figsize=(15,15))

# Random 
random = df[df['structure'].str.contains(
        "rat") == False]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = random.loc[(random['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfe_m = dfe[dfe['method'] == method] # results for this method 
    dfe_m = dfe_m.set_index('structure')
    e_res = dfe_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[0].plot(mean['energy'], label=method)
    axs[0].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[1].plot(mean['energy'], label=method)
    axs[1].legend()
    
axs[0].grid()
axs[0].set_title('Successful Random')
# axs[0].set_ylim(starty, -300)
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[0].get_ylim()

axs[1].grid()
axs[1].set_xlim(0, xticks+5)
axs[1].set_xticks(np.arange(0, xticks+5, 25))
axs[1].set_title('Failed Random')


# Rattled
rattled = df[df['structure'].str.contains(
        "rat") == True]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfe_m = dfe[dfe['method'] == method] # results for this method 
    dfe_m.set_index('structure')
    e_res = dfe_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[2].plot(mean['energy'], label=method)
    axs[2].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
axs[2].grid()
axs[2].set_title('Rattled')
axs[2].set_xlim(0, xticks+5)
axs[2].set_xticks(np.arange(0, xticks+5, 2))
starty, endy = axs[2].get_ylim()

fig.text(-0.05, 1.7, 'Energy (eV)', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Energy Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanLEnergyIterII', format=FORMAT)
plt.show()

In [ ]:
''' Mean of gnorm value per iteration '''
# %pylab

filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfg = pd.read_csv(TESTDIR+'/'+filename+'_gnorm.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

xticks = 0
fig, axs = plt.subplots(2, 1, figsize=(15,15))

# Total 
for method in methods: # for each method keep structures from both dfs

    total_m = df.loc[(df['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfg_m = dfg[dfg['method'] == method] # results for this method 
    dfg_m = dfg_m.set_index('structure')
    e_res = dfg_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['gnorm']

    axs[0].plot(mean['gnorm'], label=method)
    axs[0].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['gnorm']

    axs[1].plot(mean['gnorm'], label=method)
    axs[1].legend()
    
axs[0].grid()
axs[0].set_xticks([])
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
axs[0].set_title('Iterations of Successful Optimisation')

axs[1].grid()
axs[1].set_xlim(-5, xticks+5)
axs[1].set_xticks(np.arange(0, xticks+5, 25))
axs[1].set_title('Iterations of Failed Optimisation')

fig.text(-0.1, 1, 'Gnorm (eV)', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Gnorm Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanGnormIterI', format=FORMAT)
plt.show()

In [ ]:
''' Mean of gnorm value per iteration '''

# %pylab
filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfg = pd.read_csv(TESTDIR+'/'+filename+'_gnorm.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

fig, axs = plt.subplots(3, 1, figsize=(15,15))

# Random 
random = df[df['structure'].str.contains(
        "rat") == False]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = random.loc[(random['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfg_m = dfg[dfg['method'] == method] # results for this method 
    dfg_m = dfg_m.set_index('structure')
    e_res = dfg_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[0].plot(mean['energy'], label=method)
    axs[0].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[1].plot(mean['energy'], label=method)
    axs[1].legend()
    
axs[0].grid()
axs[0].set_title('Successful Random')
# axs[0].set_ylim(starty, -300)
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[0].get_ylim()

axs[1].grid()
axs[1].set_xlim(0, xticks+5)
axs[1].set_xticks(np.arange(0, xticks+5, 25))
# axs[1].set_ylim(starty, endy)
axs[1].set_title('Failed Random')
# axs[1].ticklabel_format(style='plain', axis='y', scilimits=(0,0))


# Rattled
rattled = df[df['structure'].str.contains(
        "rat") == True]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfg_m = dfg[dfg['method'] == method] # results for this method 
    dfg_m.set_index('structure')
    e_res = dfg_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']

    axs[2].plot(mean['energy'], label=method)
    axs[2].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
axs[2].grid()
axs[2].set_title('Rattled')
axs[2].set_xlim(0, xticks+5)
axs[2].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[2].get_ylim()

fig.text(-0.1, 1.7, 'Gnorm', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Gnorm Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanGnormIterII', format=FORMAT)
plt.show()

In [ ]:
''' Mean of gnorm value per iteration '''
''' Log scale '''

# %pylab
filename = 'iters'
df = pd.read_csv(FILENAME, skipinitialspace=True)
dfg = pd.read_csv(TESTDIR+'/'+filename+'_gnorm.csv', skipinitialspace=True)
methods = list(df['method'].unique()) # list of methods

fig, axs = plt.subplots(3, 1, figsize=(15,15))

# Random 
random = df[df['structure'].str.contains(
        "rat") == False]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = random.loc[(random['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfg_m = dfg[dfg['method'] == method] # results for this method 
    dfg_m = dfg_m.set_index('structure')
    e_res = dfg_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[0].plot(mean['energy'], label=method)
    axs[0].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
    # find failed cases
    e_fail = e_res.loc[e_res['opt_succ'] == False]
    e_fail.drop(['opt_succ'], axis=1)
    mean = e_fail.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[1].plot(mean['energy'], label=method)
    axs[1].legend()
    
axs[0].grid()
axs[0].set_title('Successful Random')
# axs[0].set_ylim(starty, -300)
axs[0].set_xlim(0, xticks+5)
axs[0].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[0].get_ylim()

axs[1].grid()
axs[1].set_xlim(0, xticks+5)
axs[1].set_xticks(np.arange(0, xticks+5, 25))
# axs[1].set_ylim(starty, endy)
axs[1].set_title('Failed Random')
# axs[1].ticklabel_format(style='plain', axis='y', scilimits=(0,0))


# Rattled
rattled = df[df['structure'].str.contains(
        "rat") == True]

xticks = 0
for method in methods: # for each method keep structures from both dfs

    total_m = rattled.loc[(rattled['method'] == method)][['structure',
                                                   'opt_succ']].set_index('structure')
    dfg_m = dfg[dfg['method'] == method] # results for this method 
    dfg_m.set_index('structure')
    e_res = dfg_m.join(total_m, on='structure') # join on each struct

    # find successful cases
    e_succ = e_res.loc[e_res['opt_succ'] == True]
    e_succ.drop(['opt_succ'], axis=1)
    mean = e_succ.groupby(['method']).mean().T # mean of each iteration in one column
    mean.columns = ['energy']
    
    mean = -mean.dropna().apply(abs).apply(np.log10) # apply log to non-NaN

    axs[2].plot(mean['energy'], label=method)
    axs[2].legend()
    lmean = len(mean.dropna())
    xticks = lmean if lmean > xticks else xticks
    
axs[2].grid()
axs[2].set_title('Rattled')
axs[2].set_xlim(0, xticks+5)
axs[2].set_xticks(np.arange(0, xticks+5, 25))
starty, endy = axs[2].get_ylim()

fig.text(-0.1, 1.7, 'Gnorm', horizontalalignment='center',
      verticalalignment='center',rotation=90, transform=plt.gca().transAxes)
fig.text(0.5, -0.15, 'Steps', horizontalalignment='center',
      verticalalignment='center', transform=plt.gca().transAxes)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Mean Gnorm Value per Iteration', x=0.524, fontsize=20)

plt.savefig(PLOTDIR+'/'+'MeanLGnormIterII', format=FORMAT)
plt.show()